In [7]:
import os
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
from cuml.cluster import HDBSCAN as cHDBSCAN
from cuml.manifold import UMAP as cUMAP
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
from bertopic.representation import MaximalMarginalRelevance
from transformers import pipeline
from bertopic.representation import TextGeneration
from sentence_transformers import SentenceTransformer

/home/arnaldo/Documents/prog-projects/complexity72_25/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
df = pd.read_parquet("./../../data/raw/cop26_tweets_en.parquet")
df.head()

,id,author_id,created_at,lang,text,retweeted_id,quoted_id,replied_id,url,expanded_url,hashtags,retweet_count,reply_count,like_count,quote_count,username,individual_or_org,category
0,1399517402984062983,40955185,2021-06-01 00:05:25+00:00,en,What is the role on @NigeriaGov at the on goin...,NaN,1.399244e+18,NaN,https://t.co/iAQsVvLDXp,https://twitter.com/COP26/status/1399244254162...,SB2021,4,1,7,0,OlumideIDOWU,Individual,Activist
1,1399525678991626240,796572560821485570,2021-06-01 00:38:18+00:00,en,Pressure is mounting on PM @ScottMorrisonMP ah...,NaN,NaN,NaN,https://t.co/iZaZWgA5wW,https://twitter.com/ActOnClimateVic/status/139...,COP26 AusClimateSolutions Auspol,21,0,15,0,ActOnClimateVic,Organization,Activist
2,1399528606540337154,16220555,2021-06-01 00:49:56+00:00,en,RT @MarkJCarney: Looking fwd to tomorrow’s fir...,1.399418e+18,NaN,NaN,,,netzero,27,0,0,0,ElizabethMay,Individual,Politics
3,1399531766344474629,202313343,2021-06-01 01:02:29+00:00,en,RT @UNFCCC: The May-June 2021 @UN Climate Chan...,1.399487e+18,NaN,NaN,,,COP26,46,0,0,0,WRIClimate,Organization,International Organization / NGO
4,1399538862158929922,796572560821485570,2021-06-01 01:30:41+00:00,en,Momentum is building in the lead up to #COP26....,NaN,NaN,NaN,https://t.co/abIgpeslsx,https://www.canberratimes.com.au/story/7277689...,COP26 AusClimateSolutions Auspol,1,0,2,0,ActOnClimateVic,Organization,Activist


In [9]:
# precompute embeddings
#embeddings = topic_model.embedding_model.encode(documents, show_progress_bar=True, )
# save embeddings
#np.save("../../data/processed/cop26_tweets_en.parquet.npy", embeddings)
embeddings = np.load('./../../data/processed/cop26_tweets_en.parquet.npy')

In [10]:
from collections import Counter

duplicates = embeddings.shape[0] - np.unique(embeddings, axis=0).shape[0]
print(f"Number of duplicate rows: {duplicates}")


Number of duplicate rows: 12998


In [11]:
unique_rows = df[df['text'].map(df['text'].value_counts()) == 1]
unique_mask = df['text'].map(df['text'].value_counts()) == 1

# Select corresponding rows from the embeddings array
unique_embeddings = embeddings[unique_mask.to_numpy()]
unique_documents = unique_rows['text']
print(len(unique_documents))


105016


In [15]:
umap_model = cUMAP(n_components=5, min_dist=0.0, metric='cosine')
#reduced_embeddings = umap_model.fit_transform(unique_embeddings)
hdbscan_model = cHDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True )
#clustering = hdbscan_model.fit_predict(reduced_embeddings)

In [12]:
import fast_hdbscan
umap_model = UMAP(n_components=5, n_neighbors=15, n_jobs=-1, metric='cosine')
hdbscan_model = fast_hdbscan.HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom')

In [ ]:
subset_documents = unique_documents.to_list()[:1000]
subset_embeddings = unique_embeddings[:1000]

In [14]:

# Initialize the BERTopic model
topic_model = BERTopic(
    vectorizer_model=CountVectorizer(ngram_range=(1, 2), stop_words='english'),
    #representation_model=KeyBERTInspired(),
    hdbscan_model = hdbscan_model,
    umap_model=umap_model,

    #hdbscan_model=HDBSCAN(min_cluster_size=100, metric='euclidean', cluster_selection_method='eom', prediction_data=False),
    verbose=True,
    calculate_probabilities=True,
    language='english',
    embedding_model=SentenceTransformer('all-MiniLM-L6-v2'),
    #dimensionality_reduction_model=BaseDimensionalityReduction(n_components=5),
    ctfidf_model=ClassTfidfTransformer(),
)


### Parallel HDBSCAN, 1K Docs
Works. It runs in 4 seconds.

In [16]:
topics, probs = topic_model.fit_transform(documents=subset_documents, embeddings=subset_embeddings)

2025-06-30 15:01:56,565 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-30 15:02:00,555 - BERTopic - Dimensionality - Completed ✓
2025-06-30 15:02:00,556 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-30 15:02:00,561 - BERTopic - Cluster - Completed ✓
2025-06-30 15:02:00,562 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-30 15:02:00,602 - BERTopic - Representation - Completed ✓


### Parallel HDBSCAN 10K Docs
It takes ~1 minute.

In [17]:
topics, probs = topic_model.fit_transform(documents=unique_documents, embeddings=unique_embeddings)

2025-06-30 15:02:32,300 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-30 15:03:18,927 - BERTopic - Dimensionality - Completed ✓
2025-06-30 15:03:18,930 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-30 15:03:19,343 - BERTopic - Cluster - Completed ✓
2025-06-30 15:03:19,359 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-30 15:03:24,025 - BERTopic - Representation - Completed ✓


### Evaluate parallel HDBSCAN
Which metric can be used?

AttributeError: 'HDBSCAN' object has no attribute 'core_dist_n_jobs'

In [8]:
import cuml
import cupy as cp
import numpy as np
from cuml.cluster import HDBSCAN
from cuml.metrics import pairwise_distances
from sklearn.metrics import silhouette_score
from sklearn.datasets import make_blobs

# Sample data (replace with your own GPU-compatible input)
X_np, _ = make_blobs(n_samples=1000, centers=5, cluster_std=1.0, random_state=42)
X = cp.asarray(X_np)

def evaluate_hdbscan(X, min_cluster_sizes, methods):
    best_score = -1
    best_params = {}
    results = []

    for min_size in min_cluster_sizes:
        for method in methods:
            try:
                clusterer = HDBSCAN(min_cluster_size=min_size,
                                    cluster_selection_method=method)
                clusterer.fit(X)
                labels = clusterer.labels_
                mask = labels != -1
                filtered_labels = cp.asnumpy(labels[mask])

                if filtered_labels.size < 2 or len(set(filtered_labels)) < 2:
                    continue


                # Compute pairwise distances (on GPU)
                # Compute pairwise distances and convert to CPU
                dists_gpu = pairwise_distances(X[mask])
                dists_cpu = dists_gpu.get()

                # Convert labels to CPU
                labels_cpu = cp.asnumpy(labels[mask])

                # Compute silhouette score
                sil_score = silhouette_score(dists_cpu, labels_cpu, metric='precomputed')

                results.append((min_size, method, sil_score))

                if sil_score > best_score:
                    best_score = sil_score
                    best_params = {
                        'min_cluster_size': min_size,
                        'cluster_selection_method': method,
                        'silhouette_score': sil_score
                    }

            except Exception as e:
                print(f"Failed for min_size={min_size}, method={method}: {e}")

    return best_params, results

# Parameter ranges
min_cluster_sizes = [15, 50, 100]
cluster_methods = ['eom', 'leaf']

best_config, all_results = evaluate_hdbscan(X, min_cluster_sizes, cluster_methods)

print("Best Parameters:")
print(best_config)
print("\nAll Results:")
for res in all_results:
    print(f"min_cluster_size={res[0]}, method={res[1]}, silhouette={res[2]:.4f}")


Best Parameters:
{'min_cluster_size': 50, 'cluster_selection_method': 'leaf', 'silhouette_score': 0.759192054001691}

All Results:
min_cluster_size=15, method=eom, silhouette=0.7300
min_cluster_size=15, method=leaf, silhouette=0.7409
min_cluster_size=50, method=eom, silhouette=0.7355
min_cluster_size=50, method=leaf, silhouette=0.7592
min_cluster_size=100, method=eom, silhouette=0.5903
min_cluster_size=100, method=leaf, silhouette=0.7443


In [ ]:
#reduced_embeddings.shape
evaluate_hdbscan(reduced_embeddings, min_cluster_sizes, cluster_methods)

Failed for min_size=15, method=eom: std::bad_alloc: out_of_memory: CUDA error (failed to allocate 11175872656 bytes) at: /tmp/pip-build-env-y7l1nuix/normal/lib/python3.12/site-packages/librmm/include/rmm/mr/device/cuda_memory_resource.hpp:62: cudaErrorMemoryAllocation out of memory


In [65]:

# Get labels and persistence scores
labels = hdbscan_model.labels_
persistence = hdbscan_model.cluster_persistence_  # shape: (n_clusters,)
persistence = np.squeeze(persistence)
valid_clusters = np.unique(labels)
valid_clusters = valid_clusters[valid_clusters != -1]  # Exclude noise (-1)
# Compute sizes of valid clusters only (in same order as persistence)
cluster_sizes = np.array([np.sum(labels == cluster_id) for cluster_id in valid_clusters])

# Now compute weighted average
weighted_persistence_score = np.average(persistence, weights=cluster_sizes)

print(f"Weighted Cluster Persistence Score: {weighted_persistence_score:.3f}")

Weighted Cluster Persistence Score: 1.000


In [1]:
import cupy as cp
from cuml.cluster import HDBSCAN

def find_best_hdbscan_params_by_noise_ratio(X, min_cluster_sizes, methods):
    best_ratio = float('inf')
    best_params = {}
    results = []

    for min_size in min_cluster_sizes:
        for method in methods:
            try:
                clusterer = HDBSCAN(min_cluster_size=min_size,
                                    cluster_selection_method=method)
                clusterer.fit(X)
                labels = clusterer.labels_

                labels_cpu = cp.asnumpy(labels)
                num_noise = (labels_cpu == -1).sum()
                unique_clusters = set(labels_cpu) - {-1}
                num_clusters = len(unique_clusters)

                if num_clusters == 0:
                    continue  # skip all-noise results

                ratio = num_noise / num_clusters
                results.append((min_size, method, num_noise, num_clusters, ratio))

                if ratio < best_ratio:
                    best_ratio = ratio
                    best_params = {
                        'min_cluster_size': min_size,
                        'cluster_selection_method': method,
                        'noise_points': num_noise,
                        'num_clusters': num_clusters,
                        'noise_to_cluster_ratio': ratio
                    }

            except Exception as e:
                print(f"Error for min_size={min_size}, method={method}: {e}")
                continue

    return best_params, results


In [ ]:
min_cluster_sizes = [5, 10, 20, 50]
cluster_methods = ['eom', 'leaf']
best_config, all_results = find_best_hdbscan_params_by_noise_ratio(reduced_embeddings, min_cluster_sizes, cluster_methods)

print("Best Parameters (by noise-to-cluster ratio):")
print(best_config)

print("\nAll Results:")
for res in all_results:
    print(f"min_cluster_size={res[0]}, method={res[1]}, noise={res[2]}, clusters={res[3]}, ratio={res[4]:.4f}")

Best Parameters (by noise-to-cluster ratio):
{'min_cluster_size': 5, 'cluster_selection_method': 'eom', 'noise_points': np.int64(50702), 'num_clusters': 2101, 'noise_to_cluster_ratio': np.float64(24.13231794383627)}

All Results:
min_cluster_size=5, method=eom, noise=50702, clusters=2101, ratio=24.1323
min_cluster_size=5, method=leaf, noise=68436, clusters=2676, ratio=25.5740
min_cluster_size=10, method=eom, noise=50413, clusters=878, ratio=57.4180
min_cluster_size=10, method=leaf, noise=68403, clusters=1107, ratio=61.7913
min_cluster_size=20, method=eom, noise=49244, clusters=399, ratio=123.4185
min_cluster_size=20, method=leaf, noise=66142, clusters=509, ratio=129.9450
min_cluster_size=50, method=eom, noise=48862, clusters=171, ratio=285.7427
min_cluster_size=50, method=leaf, noise=62244, clusters=205, ratio=303.6293


In [13]:
!pip install git+git://github.com/milesgranger/gap_statistic.git
!pip install pip install --upgrade gap-stat

from gap_statistic import OptimalK
def optimal_kmeans(data, max_k=20):
    optimalK = OptimalK(parallel_backend='joblib')
    n_clusters = optimalK(data, cluster_array=np.arange(1, max_k))
    return MiniBatchKMeans(n_clusters=n_clusters, batch_size=256).fit(data)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning git://github.com/milesgranger/gap_statistic.git to /tmp/pip-req-build-kx_r77bw
  Running command git clone --filter=blob:none --quiet git://github.com/milesgranger/gap_statistic.git /tmp/pip-req-build-kx_r77bw
ERROR: Operation cancelled by user
^C


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


ModuleNotFoundError: No module named 'gap_statistic'

In [ ]:
# pick the hdbscan and umap parameters which improve the relative_validity_ 
max_relative_validity = 0
best_params = None
for n_neighbors in [15, 50]:
    for n_components in [2, 5, 10, 20, 50]:
        umap_model = UMAP(n_neighbors=n_neighbors, n_components=n_components, min_dist=0.0, metric='cosine')
        reduced_embeddings = umap_model.fit_transform(embeddings)
        for min_cluster_size in [10, 20, 50, 100, 200, 500]:
            for cluster_selection_method in ['eom', 'leaf']:
                hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, metric='euclidean', cluster_selection_method=cluster_selection_method, prediction_data=True, gen_min_span_tree=True)
                relative_validity = hdbscan_model.fit(reduced_embeddings).relative_validity_
                if relative_validity > max_relative_validity:
                    max_relative_validity = relative_validity
                    best_params = (n_neighbors, n_components, min_cluster_size, cluster_selection_method,)
print(f"Best parameters: n_neighbors={best_params[0]}, n_components={best_params[1]}, min_cluster_size={best_params[2]}, cluster_selection_method={best_params[3]}, max_relative_validity={max_relative_validity}")
umap_model = UMAP(n_neighbors=best_params[0], n_components=best_params[1], min_dist=0.0, metric='cosine')
hdbscan_model = HDBSCAN(min_cluster_size=best_params[2], metric='euclidean', cluster_selection_method=best_params[3], prediction_data=True, gen_min_span_tree=True)

Best parameters: n_neighbors=15, n_components=50, min_cluster_size=20, cluster_selection_method=eom, max_relative_validity=0.3250034743468451


In [21]:
best_params = [15, 5, 15, 'eom']
umap_model = UMAP(n_neighbors=best_params[0], n_components=best_params[1], min_dist=0.0, metric='cosine')
hdbscan_model = HDBSCAN(min_cluster_size=best_params[2], metric='euclidean', cluster_selection_method=best_params[3], prediction_data=True, gen_min_span_tree=True)

In [ ]:

# Fit the model to the documents and embeddings
topic_model.umap_model = umap_model
topic_model.hdbscan_model = hdbscan_model
topics, probabilities = topic_model.fit_transform(documents, embeddings=embeddings)

2025-06-26 16:17:14,947 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-26 16:18:14,906 - BERTopic - Dimensionality - Completed ✓
2025-06-26 16:18:14,908 - BERTopic - Cluster - Start clustering the reduced embeddings


In [13]:
# visualize the barchart of topics
topic_model.visualize_barchart(top_n_topics=10)

In [14]:
# create a dataframe with topics
topics_df = topic_model.get_topic_info()
topics_df

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4630,-1_rt_ukraine_https_russia,"[rt, ukraine, https, russia, russian, ukrainia...",[President Biden of Russian President Vladimir...
1,0,449,0_sanctions_gas_oil_russia,"[sanctions, gas, oil, russia, energy, sanction...",[Putin continues to use energy as a weapon.\n\...
2,1,250,1_killed_children_injured_old,"[killed, children, injured, old, died, wounded...",[Prosecutor General's Office daily update: rus...
3,2,238,2_putin_rt_putin power_war,"[putin, rt, putin power, war, hope removing, s...",[@Independent My thoughts on how to penetrate ...
4,3,223,3_grain_food_global_port,"[grain, food, global, port, ports, global food...",[Russian missiles struck Odesa’s port on Satur...
...,...,...,...,...,...
63,62,23,62_losses_combat losses_estimates_indicative,"[losses, combat losses, estimates, indicative,...",[RT @KyivIndependent: These are the indicative...
64,63,23,63_russian navy_navy_sinking moskva_sinking,"[russian navy, navy, sinking moskva, sinking, ...",[@Conflicts Not surprising that the Russian Na...
65,64,23,64_rada_verkhovna rada_verkhovna_rada ukraine,"[rada, verkhovna rada, verkhovna, rada ukraine...",[Speaker of the Verkhovna Rada of #Ukraine @r_...
66,65,22,65_russian comrades_rebel russian_comrades_com...,"[russian comrades, rebel russian, comrades, co...",[@SamRamani2 I think only the Russian people c...


In [ ]:

topics_over_time = topic_model.topics_over_time(documents, timestamps,
                                                global_tuning=True, evolution_tuning=True, nr_bins=20)


In [20]:
def count_months_passed(df, col_name):


    df[col_name] = pd.to_datetime(df[col_name])

    min_date = df[col_name].min()
    max_date = df[col_name].max()

# Calculate months difference
    months_passed = (max_date.year - min_date.year) * 12 + (max_date.month - min_date.month)
    return(months_passed)
months_passed = count_months_passed(df, 'created_at')
print(f"Number of months passed: {months_passed}")


Number of months passed: 12


In [16]:
df.head()


,id,author_id,created_at,lang,in_reply_to_user_id,conversation_id,text,reply_settings,possibly_sensitive,retweeted_id,...,expanded_url,mention_name,hashtags,retweet_count,reply_count,like_count,quote_count,username,individual_or_org,category
5,1546962934294888449,1424639970,2022-07-12T21:01:19.000Z,en,,1546962934294888449,Selling drones to Russia would be a big win fo...,everyone,False,,...,https://edition.cnn.com/europe/live-news/russi...,,,6,0,33,0,IuliiaMendel,Individual,Politics
38,1546967009493139456,1424639970,2022-07-12T21:17:30.000Z,en,,1546967009493139456,"Last month, Ukrainian prosecutors launched the...",everyone,False,,...,,,,64,4,218,3,IuliiaMendel,Individual,Politics
50,1546968380602830850,1106777071,2022-07-12T21:22:57.000Z,en,1462548977367359490,1546960688899395585,@KyivIndependent My take on how Soviet hubris ...,everyone,False,,...,https://realcontextnews.com/moscows-1939-finla...,KyivIndependent,,13,4,36,0,bfry1981,Individual,Politics
53,1546968414098538499,1106777071,2022-07-12T21:23:05.000Z,en,,1546968414098538499,My take on how Soviet hubris in Finland in 193...,everyone,False,,...,https://realcontextnews.com/moscows-1939-finla...,,,0,0,1,0,bfry1981,Individual,Politics
74,1546969311130144768,1106777071,2022-07-12T21:26:39.000Z,en,4970411,1546966420046635008,@AJEnglish I've been saying for some time that...,everyone,False,,...,https://realcontextnews.com/how-ukraine-can-ta...,AJEnglish,,0,1,3,0,bfry1981,Individual,Politics


## Test to clean ukraine

In [1]:
# Load data
import pandas as pd
import numpy as np

df = pd.read_parquet("./../../data/raw/ukraine_tweets_en.parquet")
embeddings = np.load("./../../data/processed/ukraine_tweets_en.parquet.npy")
assert(len(df) == len(embeddings))
print(f"{len(df)=}")


len(df)=916955


In [2]:
# Select unique rows from dataset texts and corresponding embeddings
text_column = 'text'
unique_mask = df[text_column].map(df[text_column].value_counts()) == 1
unique_rows = df[unique_mask]
unique_embeddings = embeddings[unique_mask.to_numpy()]
assert(len(unique_rows) == len(unique_embeddings))
print(f"{len(unique_rows)=}")

len(unique_rows)=726028


In [3]:
# eliminate zero length strings
nonzero_mask = unique_rows[text_column].str.len()>0
assert(len(nonzero_mask) == len(unique_rows))
assert(len(nonzero_mask) == len(unique_embeddings))

unique_rows = unique_rows[nonzero_mask]
unique_embeddings = unique_embeddings[nonzero_mask.to_numpy()]

unique_texts = unique_rows[text_column]
assert(len(unique_rows) == len(unique_embeddings))
print(f"{len(unique_rows)=}")

len(unique_rows)=726028


## Test relative validity score from gpt

In [5]:
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score
from sklearn.cluster import HDBSCAN
import numpy as np

def approximate_relative_validity(X, clusterer):
    labels = clusterer.labels_
    mask = labels != -1

    # Require at least 2 clusters (not including noise)
    n_clusters = len(set(labels[mask]))
    if n_clusters < 2:
        return 0.0

    try:
        score = silhouette_score(X[mask], labels[mask])
        return score
    except Exception:
        return 0.0

# Generate synthetic data
X, _ = make_blobs(n_samples=3000, centers=10, cluster_std=0.5, random_state=42)

# Try several parameter settings
best_score = -1
best_clusterer = None

for min_cluster_size in [5, 10, 15, 20]:
    for min_samples in [None, 5, 10]:
        clusterer = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples).fit(X)
        score = approximate_relative_validity(X, clusterer)
        print(f"min_cluster_size={min_cluster_size}, approx validity: {score:.3f}")
        if score > best_score:
            best_score = score
            best_clusterer = clusterer

print("\nBest approx relative validity score:", best_score)

min_cluster_size=5, approx validity: 0.810
min_cluster_size=5, approx validity: 0.810
min_cluster_size=5, approx validity: 0.809
min_cluster_size=10, approx validity: 0.809
min_cluster_size=10, approx validity: 0.810
min_cluster_size=10, approx validity: 0.809
min_cluster_size=15, approx validity: 0.810
min_cluster_size=15, approx validity: 0.810
min_cluster_size=15, approx validity: 0.809
min_cluster_size=20, approx validity: 0.811
min_cluster_size=20, approx validity: 0.810
min_cluster_size=20, approx validity: 0.809

Best approx relative validity score: 0.8108249518581392


In [6]:
import fast_hdbscan
import numpy as np

best_score = -np.inf
best_params = None

for min_cluster_size in [5, 10, 20, 30]:
    for min_samples in [None, 5, 10]:
        clusterer = fast_hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,
                                         min_samples=min_samples,
                                         )
        labels = clusterer.fit_predict(X)
        if len(set(labels)) > 1:  # skip single-cluster or all-noise
            score = silhouette_score(X, labels)
            if score > best_score:
                best_score = score
                best_params = (min_cluster_size, min_samples)


In [8]:
import numpy as np
from sklearn.datasets import make_blobs
import fast_hdbscan as hdbscan # fast_hdbscan is typically imported as hdbscan
import pandas as pd # Used for convenient DataFrame operations with the MST

# --- Configuration ---
# Set a random seed for reproducibility
np.random.seed(42)

# --- 1. Generate Synthetic Data ---
# Create a dataset with distinct clusters
n_samples = 500
n_features = 2
centers = [[0, 0], [2, 2], [-2, 2], [0, -3]]
cluster_std = [0.4, 0.5, 0.5, 0.3] # Standard deviation of each cluster
X, y_true = make_blobs(n_samples=n_samples, centers=centers, cluster_std=cluster_std, random_state=42)

print(f"Generated {n_samples} samples with {n_features} features.")
print(f"Original cluster distribution (true labels): {np.unique(y_true, return_counts=True)}")

# --- 2. Apply fast_hdbscan ---
# Initialize and fit the HDBSCAN model
# min_cluster_size: The smallest size a cluster can be.
# min_samples: The number of neighbors around a point to consider it as a core point.
#              Larger values lead to more conservative clustering.
print("\nRunning HDBSCAN clustering...")
clusterer = hdbscan.HDBSCAN(min_cluster_size=15, min_samples=5, allow_single_cluster=True, store_data=True)
clusterer.fit(X)

# Get the cluster labels for each point
# -1 indicates noise points
labels = clusterer.labels_
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"HDBSCAN found {n_clusters} clusters (excluding noise).")
print(f"Cluster labels assigned: {np.unique(labels, return_counts=True)}")

# --- 3. Access the Minimum Spanning Tree (MST) ---
# The `minimum_spanning_tree_` attribute contains the MST used for hierarchy construction.
# It's a Pandas DataFrame with 'source', 'target', and 'weight' columns.
mst = clusterer.minimum_spanning_tree_

print(f"\nMinimum Spanning Tree (first 5 edges):\n{mst.head()}")
print(f"Total number of edges in MST: {len(mst)}")

# --- 4. Compute Relative Validity for each Cluster ---

def calculate_relative_validity(cluster_id, data_labels, mst_df):
    """
    Calculates a 'relative validity' score for a given cluster based on its
    internal and external MST edge weights.

    Args:
        cluster_id (int): The ID of the cluster to evaluate.
        data_labels (np.ndarray): Array of cluster labels for each data point.
        mst_df (pd.DataFrame): DataFrame representing the Minimum Spanning Tree,
                               with 'source', 'target', and 'weight' columns.

    Returns:
        float: The relative validity score for the cluster.
               Returns 0.0 if no internal or external edges are found, or in case
               of division by zero (e.g., no internal edges).
    """
    # Get indices of points belonging to the current cluster
    cluster_points_indices = np.where(data_labels == cluster_id)[0]

    if len(cluster_points_indices) == 0:
        return 0.0 # No points in this cluster

    internal_edge_weights = []
    external_edge_weights = []

    for _, row in mst_df.iterrows():
        source = row['source']
        target = row['target']
        weight = row['weight']

        is_source_in_cluster = source in cluster_points_indices
        is_target_in_cluster = target in cluster_points_indices

        if is_source_in_cluster and is_target_in_cluster:
            # Both points are in the current cluster -> Internal edge
            internal_edge_weights.append(weight)
        elif is_source_in_cluster != is_target_in_cluster:
            # One point is in the cluster, the other is outside -> External edge
            external_edge_weights.append(weight)

    max_internal_weight = 0.0
    if internal_edge_weights:
        max_internal_weight = max(internal_edge_weights)
    else:
        # If there are no internal edges (e.g., singleton cluster or very sparse connectivity),
        # consider its internal "tightness" as infinite, leading to a validity of 0 or undefined.
        # For this metric, we'll assign a very small value to avoid division by zero
        # or indicate poor internal cohesion.
        # A cluster without internal edges (connecting its own members) is poorly formed.
        print(f"  Warning: Cluster {cluster_id} has no internal MST edges. Max internal weight set to a small value.")
        max_internal_weight = 1e-9 # Prevent division by zero, indicates poor internal cohesion

    min_external_weight = float('inf')
    if external_edge_weights:
        min_external_weight = min(external_edge_weights)
    else:
        # If there are no external edges, the cluster is perfectly isolated.
        # This is a very strong cluster, so assign a very high value.
        print(f"  Info: Cluster {cluster_id} has no external MST edges. Min external weight set to infinity.")
        return float('inf')

    # Calculate relative validity
    # A higher score means better separation (larger min_external)
    # and tighter internal cohesion (smaller max_internal).
    relative_validity = min_external_weight / max_internal_weight

    return relative_validity

print("\n--- Calculating Relative Validity for each cluster ---")
relative_validity_scores = {}
for cluster_id in sorted(np.unique(labels)):
    if cluster_id == -1:
        # Skip noise points as they don't form a coherent cluster
        continue

    score = calculate_relative_validity(cluster_id, labels, mst)
    relative_validity_scores[cluster_id] = score
    print(f"Cluster {cluster_id}: Relative Validity = {score:.4f}")

print("\n--- Summary of Relative Validity Scores ---")
if relative_validity_scores:
    for cluster_id, score in sorted(relative_validity_scores.items()):
        print(f"Cluster {cluster_id}: {score:.4f}")
else:
    print("No clusters found (or only noise).")

# --- Visualizing MST and Clusters (Optional, for better understanding) ---
# This part requires matplotlib to be installed.
try:
    import matplotlib.pyplot as plt
    from scipy.sparse import coo_matrix
    from scipy.sparse.csgraph import minimum_spanning_tree

    print("\n--- Visualizing MST and Clusters ---")

    plt.figure(figsize=(10, 8))

    # Plot data points, colored by cluster
    unique_labels = np.unique(labels)
    colors = plt.cm.get_cmap('Spectral', len(unique_labels))
    for k, col in zip(unique_labels, colors(np.linspace(0, 1, len(unique_labels)))):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)
        xy = X[class_member_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=6, label=f'Cluster {k}')

    # Plot MST edges
    # Filter MST edges for visualization (e.g., only show edges with weight below a threshold)
    # This helps to avoid clutter for very large datasets
    # You might want to adjust this threshold or remove it based on your data
    # For simplicity, let's plot all edges for now
    for _, row in mst.iterrows():
        source_idx = row['source']
        target_idx = row['target']
        weight = row['weight']

        # Get coordinates of source and target points
        p1 = X[source_idx]
        p2 = X[target_idx]

        # Draw a line between them
        # You can color-code edges if you want to distinguish internal/external
        plt.plot([p1[0], p2[0]], [p1[1], p2[1]], 'grey', linewidth=0.5, alpha=0.6)


    plt.title('HDBSCAN Clustering with MST Edges')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

except ImportError:
    print("\nMatplotlib not installed. Skipping visualization.")
    print("To install: pip install matplotlib")



Generated 500 samples with 2 features.
Original cluster distribution (true labels): (array([0, 1, 2, 3]), array([125, 125, 125, 125]))

Running HDBSCAN clustering...
HDBSCAN found 4 clusters (excluding noise).
Cluster labels assigned: (array([-1,  0,  1,  2,  3]), array([  4, 125, 125, 124, 122]))


AttributeError: 'MinimumSpanningTree' object has no attribute 'head'

In [2]:
import fast_hdbscan as fast_hdbscan
import hdbscan as hdbscan
import numpy as np
from sklearn.datasets import make_blobs

X = np.load('./../../data/processed/cop26_tweets_en.parquet.npy')
X = X[:1000]


data, _ = make_blobs(1000)



In [41]:
best_score = -np.inf
best_params = None
#X_reduced = umap_model.fit(X)
for min_cluster_size in [5, 10, 20, 30]:
    for cluster_selection_method in ['leaf','eom']:
        clusterer = fast_hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,
                                         cluster_selection_method=cluster_selection_method,
                                         )
        labels = clusterer.fit_predict(data)

In [1]:
from hdbscan import validity_index
labels = clusterer.fit_predict(data)
validity_index(data, labels)

NameError: name 'clusterer' is not defined

In [51]:
X_reduced = umap_model.fit_transform(X)
best_validity = -float('Inf')
X_reduced.shape
#clusterer.fit_predict(X_reduced)

(1000, 5)

In [76]:
best_params = {}
best_validity = -float('Inf')

for n_components in [5,15,30, 50]:
    for n_neighbors in [15, 50]:
        reduction_model = UMAP(n_neighbors=n_neighbors, n_components=n_neighbors, metric='cosine', n_jobs=-1)
        X_reduced = reduction_model.fit_transform(X)
        for min_cluster_size in [10, 15, 30]:
            for cluster_selection_method in ['leaf','eom']:
                clusterer = fast_hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,
                                            cluster_selection_method=cluster_selection_method,
                                            )
                labels = clusterer.fit_predict(X_reduced)
                noise = len(np.where(labels==-1)[0])/len(labels)
                n_clusters = len(np.unique(labels))-1

                curr_validity = validity_index(X_reduced.astype(np.float64), labels, metric='euclidean')
                print(f'{min_cluster_size=}, {cluster_selection_method=},{curr_validity=}, {noise=}, {n_clusters=}')
                if best_validity < curr_validity:
                    best_validity = curr_validity
                    best_params['n_neighbors'] = n_neighbors
                    best_params['n_components'] = n_components
                    best_params['min_cluster_size'] = min_cluster_size
                    best_params['cluster_selection_method'] = cluster_selection_method
noise = len(np.where(labels==-1)[0])/len(labels)
n_clusters = len(np.unique(labels))-1
print(f"Best Parameters: {best_params}\nBest Validity: {best_validity}")
print(f"{noise=}, {n_clusters=}")

min_cluster_size=10, cluster_selection_method='leaf',curr_validity=np.float64(0.4280920920455761), noise=0.271, n_clusters=29
min_cluster_size=10, cluster_selection_method='eom',curr_validity=np.float64(0.42473240397814277), noise=0.247, n_clusters=27
min_cluster_size=15, cluster_selection_method='leaf',curr_validity=np.float64(0.3706763289696604), noise=0.389, n_clusters=20
min_cluster_size=15, cluster_selection_method='eom',curr_validity=np.float64(0.8502696934575713), noise=0.0, n_clusters=2
min_cluster_size=30, cluster_selection_method='leaf',curr_validity=np.float64(0.12244389530367211), noise=0.726, n_clusters=5
min_cluster_size=30, cluster_selection_method='eom',curr_validity=np.float64(0.25436919508650147), noise=0.406, n_clusters=2
min_cluster_size=10, cluster_selection_method='leaf',curr_validity=np.float64(0.291613516912397), noise=0.462, n_clusters=24
min_cluster_size=10, cluster_selection_method='eom',curr_validity=np.float64(0.6589600258299887), noise=0.0, n_clusters=2
mi

noise=0.676, n_clusters=5


In [ ]:
best_score = -np.inf
best_params = None

for min_cluster_size in [5, 10, 20, 30]:
    for cluster_selection_method in ['leaf','eom']:
        clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,
                                         cluster_selection_method=cluster_selection_method,
                                         )
        labels = clusterer.fit_predict(X)

/home/arnaldo/Documents/prog-projects/complexity72_25/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/arnaldo/Documents/prog-projects/complexity72_25/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/arnaldo/Documents/prog-projects/complexity72_25/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/arnaldo/Documents/prog-projects/complexity72_25/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/arnaldo/Documents/

In [24]:
from sklearn import cluster
from hdbscan import validity_index
import numpy as np
from umap import UMAP
import random
my_random_state = random.randint(0,2**32-1) # 1138341792
best_score = -np.inf
best_params = None
reducer = UMAP(n_neighbors=15, n_components=5, metric='cosine', n_jobs=-1, min_dist=0.0, random_state=my_random_state)
X_reduced = reducer.fit_transform(X)
print(f"{my_random_state=}")
for min_cluster_size in [5, 10, 20, 30]:
    for cluster_selection_method in ['leaf','eom']:
        print(f'{min_cluster_size=}, {cluster_selection_method=}')
        clusterer = cluster.HDBSCAN(min_cluster_size=min_cluster_size,
                                         cluster_selection_method=cluster_selection_method,
                                         n_jobs = -1,
                                         )
        labels = clusterer.fit_predict(X_reduced)
        #this_validity = validity_index(X.astype(np.float64), labels)
        this_reduced_validity = validity_index(X_reduced.astype(np.float64), labels)
        
        #print(f"{this_validity}, {this_reduced_validity}")
        print(f"{this_reduced_validity=}")

/home/arnaldo/Documents/prog-projects/complexity72_25/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/arnaldo/Documents/prog-projects/complexity72_25/.venv/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


my_random_state=3640738295
min_cluster_size=5, cluster_selection_method='leaf'
this_reduced_validity=np.float64(0.49945804304443414)
min_cluster_size=5, cluster_selection_method='eom'
this_reduced_validity=np.float64(0.5108110361457783)
min_cluster_size=10, cluster_selection_method='leaf'
this_reduced_validity=np.float64(0.5130051115813761)
min_cluster_size=10, cluster_selection_method='eom'
this_reduced_validity=np.float64(0.5066864441829844)
min_cluster_size=20, cluster_selection_method='leaf'
this_reduced_validity=np.float64(0.3485894451356984)
min_cluster_size=20, cluster_selection_method='eom'
this_reduced_validity=np.float64(0.3657448485834856)
min_cluster_size=30, cluster_selection_method='leaf'
this_reduced_validity=np.float64(0.16511476104767048)
min_cluster_size=30, cluster_selection_method='eom'
this_reduced_validity=np.float64(0.32336811056224557)
